In [1]:
import re
from pathlib import Path
from typing import List, Dict, Any

# Example signatures
SIGNATURES = [
    r"eval\(",
    r"base64\.b64decode",
    r"socket\.connect",
    r"exec\(",
    r"import\s+os"
]

# Compile regexes for speed (case-insensitive where helpful)
COMPILED = [re.compile(pat, re.IGNORECASE) for pat in SIGNATURES]


In [2]:
def scan_file_for_signatures(path: Path, compiled_patterns: List[re.Pattern], context_chars: int = 40):
   
    matches = []
    try:
        raw = path.read_bytes()
    except Exception as e:
        # unreadable file (permissions etc.)
        return {"error": f"could not read file: {e}", "matches": []}
    
    # Try decode safely: utf-8 else latin-1 fallback
    try:
        text = raw.decode("utf-8")
    except UnicodeDecodeError:
        try:
            text = raw.decode("latin-1")
        except Exception:
            return {"error": "could not decode file", "matches": []}
    
    for pat in compiled_patterns:
        for m in pat.finditer(text):
            start, end = m.start(), m.end()
            snippet_start = max(0, start - context_chars)
            snippet_end = min(len(text), end + context_chars)
            snippet = text[snippet_start:snippet_end].replace("\n", "␤")
            matches.append({
                "pattern": pat.pattern,
                "match_text": m.group(0),
                "snippet": snippet,
                "start": start,
                "end": end
            })
    return {"error": None, "matches": matches}


In [3]:
def scan_directory(root_dir: str, compiled_patterns: List[re.Pattern],
                   include_ext: List[str] = None, exclude_dirs: List[str] = None,
                   verbose: bool = True) -> Dict[str, Any]:
   
    root = Path(root_dir).resolve()
    report = {"matches": {}, "scanned_files": 0, "matched_files": 0}
    exclude_dirs = set(d.lower() for d in (exclude_dirs or []))
    
    for p in root.rglob("*"):
        if p.is_file():
            # Skip files in excluded dirs
            if any(part.lower() in exclude_dirs for part in p.parts):
                continue
            # Filter by extension when provided
            if include_ext is not None:
                if p.suffix.lower() not in include_ext:
                    continue
            report["scanned_files"] += 1
            result = scan_file_for_signatures(p, compiled_patterns)
            if result.get("error"):
                # optionally log or skip
                continue
            if result["matches"]:
                rel = str(p.relative_to(root))
                report["matches"][rel] = result["matches"]
                report["matched_files"] += 1
    
    if verbose:
        print(f"🔎 Scan complete for: {root}")
        print(f"Files scanned: {report['scanned_files']}")
        print(f"Files matched: {report['matched_files']}")
        if report["matched_files"] == 0:
            print("✅ No suspicious patterns found.")
        else:
            print("\nSuspicious findings:")
            for rel, matches in report["matches"].items():
                print(f"\n-- {rel}  ({len(matches)} matches)")
                for i, m in enumerate(matches, 1):
                    print(f"  {i}) pattern: {m['pattern']}  match: {m['match_text']}")
                    print(f"     snippet: ...{m['snippet']}...")
    return report


In [ ]:
demo_root = "integrity_demo"  


script_exts = ['.py', '.js', '.sh', '.ps1', '.bat', '.vbs']

report = scan_directory(demo_root, COMPILED, include_ext=script_exts, exclude_dirs=['.git', '__pycache__'])
